In [1]:
import pandas as pd
import altair as alt
alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

In [2]:
#filtered_df = pd.read_csv('../data/processed/processed_data.csv')

In [3]:
#df_no_uk = filtered_df[filtered_df['Country'] != 'United Kingdom']

In [4]:
#df_no_uk = filtered_df[filtered_df['Country'] != 'United Kingdom']
#country_counts = df_no_uk['Country'].value_counts().reset_index()
#country_counts.columns = ['Country', 'Count']
#country_counts['Percentage'] = round((country_counts['Count'] / country_counts['Count'].sum()) * 100,0)
#country_counts = country_counts.head(5)
#country_counts.to_csv('../data/filtered/pie_chart_data.csv', index=False)
#country_counts = pd.read_csv('../data/filtered/pie_chart_data.csv')
#country_counts

In [5]:
#pie_chart = alt.Chart(country_counts).mark_arc().encode(
#    theta=alt.Theta(field="Percentage", type="quantitative"),  # Use Percentage for the arc size
#    color=alt.Color(field="Country", type="nominal", 
#                    scale=alt.Scale(scheme='pastel1')),  # Use a categorical color scheme
#    tooltip=['Country', 'Percentage']  # Show Percentage in the tooltip
#).properties(
#    title="Top 5 Countries Outside of the UK"
#)
#pie_chart

In [65]:
def plot_waterfall_chart(data):

    # Compute Gross Revenue (sum of revenue where quantity > 0)
    gross_revenue = data.loc[data['Quantity'] > 0, 'Revenue'].sum()

    # Compute Refund (sum of revenue where quantity < 0)
    refund = data.loc[data['Quantity'] < 0, 'Revenue'].sum()

    # Compute Net Revenue (Gross Revenue + Refund)
    net_revenue = gross_revenue + refund

    # Create the final DataFrame
    df = pd.DataFrame({
        'Category': ['Gross Revenue', 'Refund', 'Net Revenue'],
        'Value': [gross_revenue, refund, net_revenue]
    })


    # Define explicit category order
    category_order = df['Category'].tolist()
    
    # Add an index column to enforce order
    df['Index'] = range(len(df))  # Assign numerical order explicitly

    # Convert Category column to categorical with correct order
    df['Category'] = pd.Categorical(df['Category'], categories=category_order, ordered=True)

    # Compute cumulative values properly
    df['Start'] = df['Value'].cumsum().shift(1).fillna(0)
    df['End'] = df['Start'] + df['Value']

    # Force Net Revenue to start from zero
    df.loc[df['Category'] == 'Net Revenue', 'Start'] = 0
    df.loc[df['Category'] == 'Net Revenue', 'End'] = df['Value']

    # Define colors
    df['Color'] = df['Value'].apply(lambda x: 'Increase' if x > 0 else 'Decrease')

    # Create a mapping of Index to Category labels
    category_labels = {i: cat for i, cat in enumerate(df['Category'])}

    # Create the waterfall chart
    bars = alt.Chart(df).mark_bar().encode(
        x=alt.X('Index:O', title='Category', sort=list(df['Index']),  
                axis=alt.Axis(labelExpr=f"datum.value == 0 ? '{category_labels[0]}' : " +
                                       f"datum.value == 1 ? '{category_labels[1]}' : " +
                                       f"datum.value == 2 ? '{category_labels[2]}' : ''",
                              labelAngle=-45)),  # Replaces 0,1,2 with actual labels
        y=alt.Y('Start:Q', title='Revenue'),
        y2='End:Q',
        color=alt.Color('Color:N', scale=alt.Scale(domain=['Increase', 'Decrease'], range=['green', 'red']), legend=None),
        tooltip=['Category', 'Value']
    )

    # Add text labels
    text = alt.Chart(df).mark_text(dy=-10, size=12).encode(
        x='Index:O',
        y='End:Q',
        text=alt.Text('Value:Q', format=',.0f')
    )

    # Combine bars and labels
    waterfall_chart = (bars + text).properties(width=600, height=400, title="Revenue Waterfall Chart" )
    
    return waterfall_chart



In [66]:
data = pd.read_csv('../data/processed/processed_data.csv')
plot_waterfall_chart(data)


alt.LayerChart(...)